# AWS Sagemaker XGBoost Demo

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} 
               # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " 
      + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-2 region. You will use the 825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [3]:
# create s3 bucket
bucket_name = 'sagemaker-tutorial-07911644521'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


### Data Preprocessing

In [4]:
# download clenaed bank dataset
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [6]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   age                            41188 non-null  int64
 1   campaign                       41188 non-null  int64
 2   pdays                          41188 non-null  int64
 3   previous                       41188 non-null  int64
 4   no_previous_contact            41188 non-null  int64
 5   not_working                    41188 non-null  int64
 6   job_admin.                     41188 non-null  int64
 7   job_blue-collar                41188 non-null  int64
 8   job_entrepreneur               41188 non-null  int64
 9   job_housemaid                  41188 non-null  int64
 10  job_management                 41188 non-null  int64
 11  job_retired                    41188 non-null  int64
 12  job_self-employed              41188 non-null  int64
 13  job_services    

In [7]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [9]:
model_data['pdays'].value_counts()

999    39673
3        439
6        412
4        118
9         64
2         61
7         60
12        58
10        52
5         46
13        36
11        28
1         26
15        24
14        20
8         18
0         15
16        11
17         8
18         7
19         3
22         3
21         2
26         1
20         1
25         1
27         1
Name: pdays, dtype: int64

In [10]:
# train/test split data
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])

train_data.shape, test_data.shape

((28831, 61), (12357, 61))

In [11]:
# reformats the header and first column of the training data
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], 
          axis=1).to_csv('train.csv', index=False, header=False)

# save training data to csv
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

### Modeling

In [12]:
sess = sagemaker.Session()

# instantiate xgboost model
xgb = sagemaker.estimator.Estimator(containers[my_region],role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket_name, prefix),
                                    sagemaker_session=sess)

# set model hyperparameters
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

In [13]:
# fit model
xgb.fit({'train': s3_input_train})

2021-02-21 20:14:29 Starting - Starting the training job...
2021-02-21 20:14:51 Starting - Launching requested ML instancesProfilerReport-1613938468: InProgress
......
2021-02-21 20:15:52 Starting - Preparing the instances for training......
2021-02-21 20:16:53 Downloading - Downloading input data
2021-02-21 20:16:53 Training - Downloading the training image..Arguments: train
[2021-02-21:20:17:07:INFO] Running standalone xgboost training.
[2021-02-21:20:17:07:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-02-21:20:17:07:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8423.11mb
[2021-02-21:20:17:07:INFO] Determined delimiter of CSV input is ','
[20:17:07] S3DistributionType set as FullyReplicated
[20:17:07] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[20:17:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 14 pruned nodes, max_depth=

In [14]:
# deploy model predictions
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

-------------!

In [15]:
from sagemaker.serializers import CSVSerializer

#load the data into an array
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values 
# set the serializer type
xgb_predictor.serializer = CSVSerializer() 
# predict!
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
# and turn the prediction into an array
predictions_array = np.fromstring(predictions[1:], sep=',') 
print(predictions_array.shape)

(12357,)


### Evaluating Model Performance

In [16]:
cm = pd.crosstab(index=test_data['y_yes'], 
                 columns=np.round(predictions_array), 
                 rownames=['Observed'], 
                 colnames=['Predicted'])

tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100

print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10785)    35% (151)
Purchase        10% (1143)     65% (278) 



### Clean Up

In [17]:
# delete endpoints
xgb_predictor.delete_endpoint()
xgb_predictor.delete_model()

In [18]:
# delete training artifacts and s3 bucket
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '27BSZ52JP3JDJAFQ',
   'HostId': 'WXFAjYQk7PlrmKkrGDRU2GKbfkMMQvIziiESgFmZdxpGdYDEpdVt0LoXnGvRdnuokxO3qGyGXAc=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'WXFAjYQk7PlrmKkrGDRU2GKbfkMMQvIziiESgFmZdxpGdYDEpdVt0LoXnGvRdnuokxO3qGyGXAc=',
    'x-amz-request-id': '27BSZ52JP3JDJAFQ',
    'date': 'Sun, 21 Feb 2021 20:25:22 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-02-21-20-14-28-961/profiler-output/framework/training_job_end.ts'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-02-21-20-14-28-961/rule-output/ProfilerReport-1613938468/profiler-output/profiler-reports/GPUMemoryIncrease.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-02-21-20-14-28-961/profiler-output/system/training_job_end.ts'},
   {'Key': 'sagemaker/DEMO-xgboost-d